# 두번째 베이스라인

이전의 베이스라인은 잘 보셨나요?

이전에는 단순히 날짜를 년, 월, 일로 분류한 데이터를 추가하여 예측을 진행했는데요,

이번에는 좀 더 변수들을 이리저리 조합해서 데이터 분석을 진행해보겠습니다.

이 과정을 Feature Engineering 이라고 합니다.

데이터 분석에서는 빠질 수 없는 매우 중요한 과정이에요.

동일한 모델을 사용해도 어떤 데이터를 투입하는가에 따라 성능이 얼마나 달라질 수 있는지 확인해봐요!

## 데이터 불러오기

우선 학습에 필요한 train데이터를 불러오겠습니다.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

bicycle = pd.read_csv('data/train.csv')
bicycle.head()

,date_time,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
0,2018-04-01,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,22994
1,2018-04-02,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,28139
2,2018-04-03,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,26817
3,2018-04-04,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,26034
4,2018-04-05,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,2833


## 일자 분리

모델에 수치를 넣고 학습시키기 위해선 글자나 문자를 숫자로 바꿔주는 작업이 필요합니다.

주어진 데이터를 살펴보면 date_time부분이 문자열(string)로 되어있는 것을 확인하실 수 있습니다.

이를 각각 년도, 월, 일로 나누어 숫자형으로 변환해 보겠습니다.

In [2]:
def seperate_datetime(dataframe):
    year = []
    month = []
    day = []

    for date in dataframe.date_time:
        year_point, month_point, day_point = date.split('-') # - 기준으로 string을 나누고 list로 만듦 ex) '2016-04-01' -> ['2016', '04', '01']
        year.append(int(year_point)-2017)
        month.append(int(month_point)-3)
        day.append(int(day_point))
    return year, month, day

year, month, day = seperate_datetime(bicycle)

bicycle['year'] = year
bicycle['month'] = month
bicycle['day'] = day

bicycle.head()

,date_time,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals,year,month,day
0,2018-04-01,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,22994,1,1,1
1,2018-04-02,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,28139,1,1,2
2,2018-04-03,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,26817,1,1,3
3,2018-04-04,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,26034,1,1,4
4,2018-04-05,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,2833,1,1,5


년도, 월, 일이 전부 잘 분리되어 들어간게 보이시나요?

이제 학습에 필요한 X와 y를 분리해 보겠습니다

## 변수 및 모델 정의

우리의 목적은 주어진 날씨 데이터(low_temp, high_temp, precipitaion, wind_speed, humidity, insolation, cloud, year, month, day)를 이용하여 사람들이 따릉이를 얼마나 빌리는지(number_of_rentals) 예측하는 것입니다.

그럼 날씨 데이터가 X, 따릉이 대여 수가 y가 되겠죠?

In [3]:
X  = bicycle.drop(['date_time', 'number_of_rentals'], axis=1)
y = bicycle.number_of_rentals
X

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,1
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,2
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,3
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,4
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...
268,228.662,3.980,0.223,2.271,78.378,20.500,27.526,36.486,3,3,26
269,207.770,2.865,0.081,1.794,78.412,20.812,28.842,21.081,3,3,27
270,282.568,1.730,0.000,1.820,72.736,21.000,29.053,7.297,3,3,28
271,137.027,2.257,0.088,2.043,70.473,19.625,26.000,15.541,3,3,29


# 0. original model 성능 확인

Feature Engineering에 따른 성능 비교를 위해 먼저 아무런 처리를 하지 않은 original 모델의 성능을 확인해보겠습니다.

In [4]:
# metric 정의

import numpy as np

def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score

In [5]:
from sklearn.linear_model import LinearRegression

model = LinearRegression() # 모델 정의
model.fit(X, y) # 학습

y_hat = model.predict(X) # y 예측

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')

모델 NMAE: 0.32347975212014557


오차율이 32% 정도네요!

빈말로도 잘 맞춘다고는 하기 힘들 것 같습니다...

# 1. Feature Engineering by Human

이제 본격적으로 Feature Engineering 을 시작해보겠습니다.

Feature Engineering은 데이터를 분석하기 위해 필요하다고 생각하는 정보를 만들어서 추가해주는 작업입니다.

먼저 사람이 직접 보면서 필요한 정보가 무엇일지 추론하고, 그 정보를 추가해봅시다.

## 요일 정보 추가

따릉이를 이용하는 사람들 중에는 휴일에 놀러가기 위해 이용하는 사람들도 있을 것입니다.

이를 고려하면 따릉이 이용자 수를 예측하는 데에 요일 정보가 도움이 될 것입니다.

In [6]:
# 1번은 사람 손으로 만든 feature들을 사용합니다.
X_human = X.copy()

In [7]:
from sklearn.preprocessing import LabelEncoder

# 요일 정보 추가
week_day = pd.to_datetime(bicycle['date_time']).dt.day_name()
le = LabelEncoder()
le.fit(week_day)
X_human['week_day'] = le.transform(week_day)
X_human.head()

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day,week_day
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,1,3
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,2,1
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,3,5
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,4,6
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,5,4


## 온도 정보 추가

밖이 너무 춥거나, 너무 덥다면 밖에 나가고 싶지 않겠죠?

따릉이 이용자도 마찬가지로 날씨가 따릉이를 타기에 적절하지 않다면 대중교통이나 택시를 이용할 것입니다.

이 인사이트를 바탕으로

1. 일교차가 너무 큰 날씨
2. 너무 덥고 습한 날씨
3. 춥고 바람부는 날씨

위 3가지 날씨에는 따릉이 이용자 수가 적을 것이라는 가정을 세워봅시다.

이 가정이 맞는지 확인하기 위해서는 어떤 정보들이 필요할까요?

아래 3가지는 어떤가요?

In [8]:
# 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
X_human['temp_diff_info'] = X_human['high_temp'] - X_human['low_temp']

# 2. "덥고 습한 날씨"를 알기 위한 정보
X_human['sweat_info'] = X_human['high_temp'] * X_human['humidity'] 

# 3. "춥고 바람부는 날씨"를 알기 위한 정보
X_human['cold_info'] = X_human['low_temp'] * X_human['wind_speed'] 

X_human.head()

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day,week_day,temp_diff_info,sweat_info,cold_info
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,1,3,8.400,1575.000000,38.430000
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,2,1,6.188,1326.827000,41.997736
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,3,5,5.004,1146.846764,27.739280
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,4,6,4.056,888.628432,26.083056
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,5,4,4.546,768.903064,18.717750


이렇게 새로운 feature를 추가했으니 이제 성능이 어떻게 변하는지 살펴봐야죠!

In [9]:
## 1번 모델 성능 확인

model = LinearRegression() # 모델 정의
model.fit(X_human, y) # 학습

y_hat = model.predict(X_human) # y 예측

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')

모델 NMAE: 0.2842626127833205


예측 오차율에서 약 4%p 정도의 성능 향상이 일어난 것을 확인할 수 있습니다.

단순히 두 feature를 곱한 추가 feature를 4개 생성했을 뿐인데 말이에요!

# 2. Feature Engineering by Computer

이번에는 사람의 판단이 들어가지 않고 오로지 컴퓨터의 단순 계산만을 이용해서 feature를 추가해보겠습니다.

sklearn 등에서 제공하는 자동 feature engineering이 이런 방식입니다.

단순히 두 변수간의 곱을 추가해준 다음, 성능이 어떻게 변하는지 살펴봅시다.

In [10]:
# 2번은 컴퓨터로 만든 feature들을 사용합니다.
X_computer = X.copy()

In [11]:
col_list = X_computer.columns

# 이중 for문을 사용하여 feature 자기 자신의 제곱과 두 feature간의 곱이라는 새로운 feature를 추가합니다.
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        X_computer[f'{col_list[i]}*{col_list[j]}'] = X_computer[col_list[i]] * X_computer[col_list[j]]

X_computer

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,...,Precipitation_Probability*Precipitation_Probability,Precipitation_Probability*year,Precipitation_Probability*month,Precipitation_Probability*day,year*year,year*month,year*day,month*month,month*day,day*day
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,...,900.000000,30.000,30.000,30.000,1,1,1,1,1,1
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,...,380.250000,19.500,19.500,39.000,1,1,2,1,2,4
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,...,365.306769,19.113,19.113,57.339,1,1,3,1,3,9
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,...,1891.641049,43.493,43.493,173.972,1,1,4,1,4,16
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,...,4016.770884,63.378,63.378,316.890,1,1,5,1,5,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,228.662,3.980,0.223,2.271,78.378,20.500,27.526,36.486,3,3,...,1331.228196,109.458,109.458,948.636,9,9,78,9,78,676
269,207.770,2.865,0.081,1.794,78.412,20.812,28.842,21.081,3,3,...,444.408561,63.243,63.243,569.187,9,9,81,9,81,729
270,282.568,1.730,0.000,1.820,72.736,21.000,29.053,7.297,3,3,...,53.246209,21.891,21.891,204.316,9,9,84,9,84,784
271,137.027,2.257,0.088,2.043,70.473,19.625,26.000,15.541,3,3,...,241.522681,46.623,46.623,450.689,9,9,87,9,87,841


원래 11개만 있었던 feature가 무려 77개로 늘어났네요.

그럼 이번 feature engineering의 성능은 어떨까요?

In [12]:
## 2번 모델 성능 확인

model = LinearRegression() # 모델 정의
model.fit(X_computer, y) # 학습

y_hat = model.predict(X_computer) # y 예측

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')

모델 NMAE: 0.14013355728774618


무려 기존 모델 대비 오차율이 18%p 줄어든 압도적인 성능 향상이 일어난 것을 확인할 수 있습니다.

1번보다도 14%p 더 좋은 성능을 보여주네요.

그렇다면 컴퓨터를 이용해서 자동으로 생성되는 feature를 사용하면 사람이 데이터를 직접 보는 것보다 더 편하게, 더 좋은 성능의 모델을 찾을 수 있는 걸까요?

데이터 분석에 더이상 사람은 필요없는 것일까요?

# 3. Feature Engineering by Human & Computer

이를 알아보기 위해 이번에는 1번에서 사람이 추가한 정보와 2번의 방식을 혼용해보겠습니다.

In [13]:
# 3번은 2번에 사람의 도메인 지식을 추가한 형태입니다.
X_combination = X.copy()

In [14]:
from sklearn.preprocessing import LabelEncoder

# 요일 정보 추가
week_day = pd.to_datetime(bicycle['date_time']).dt.day_name()
le = LabelEncoder()
le.fit(week_day)
X_combination['week_day'] = le.transform(week_day)

# 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
X_combination['temp_diff_info'] = X_combination['high_temp'] - X_combination['low_temp']

# 2. "덥고 습한 날씨"를 알기 위한 정보
X_combination['sweat_info'] = X_combination['high_temp'] * X_combination['humidity'] 

# 3. "춥고 바람부는 날씨"를 알기 위한 정보
X_combination['cold_info'] = X_combination['low_temp'] * X_combination['wind_speed'] 

In [15]:
col_list = X_combination.columns

# 이중 for문을 사용하여 변수 자기 자신의 제곱과 두 변수간의 곱이라는 새로운 변수를 추가합니다.
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        X_combination[f'{col_list[i]}*{col_list[j]}'] = X_combination[col_list[i]] * X_combination[col_list[j]]

X_combination

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,...,week_day*week_day,week_day*temp_diff_info,week_day*sweat_info,week_day*cold_info,temp_diff_info*temp_diff_info,temp_diff_info*sweat_info,temp_diff_info*cold_info,sweat_info*sweat_info,sweat_info*cold_info,cold_info*cold_info
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,...,9,25.200,4725.000000,115.290000,70.560000,13230.000000,322.812000,2.480625e+06,60527.250000,1476.864900
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,...,1,6.188,1326.827000,41.997736,38.291344,8210.405476,259.881990,1.760470e+06,55723.730064,1763.809829
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,...,25,25.020,5734.233820,138.696400,25.040016,5738.821207,138.807357,1.315258e+06,31812.703504,769.467655
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,...,36,24.336,5331.770592,156.498336,16.451136,3604.276920,105.792875,7.896605e+05,23178.145155,680.325810
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,...,16,18.184,3075.612256,74.871000,20.666116,3495.433329,85.090891,5.912119e+05,14392.135326,350.354165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,228.662,3.980,0.223,2.271,78.378,20.500,27.526,36.486,3,3,...,0,0.000,0.000000,0.000000,49.364676,15158.123050,327.098943,4.654516e+06,100440.364024,2167.414580
269,207.770,2.865,0.081,1.794,78.412,20.812,28.842,21.081,3,3,...,4,16.060,4523.117808,74.673456,64.480900,18160.317999,299.813926,5.114649e+06,84439.209655,1394.031258
270,282.568,1.730,0.000,1.820,72.736,21.000,29.053,7.297,3,3,...,9,24.159,6339.597024,114.660000,64.850809,17017.591611,307.785660,4.465610e+06,80766.466086,1460.768400
271,137.027,2.257,0.088,2.043,70.473,19.625,26.000,15.541,3,3,...,1,6.375,1832.298000,40.093875,40.640625,11680.899750,255.598453,3.357316e+06,73463.926975,1607.518813


In [16]:
## 3번 모델 성능 확인

model = LinearRegression() # 모델 정의
model.fit(X_combination, y) # 학습

y_hat = model.predict(X_combination) # y 예측

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')

모델 NMAE: 0.10278949918042468


기존 모델 대비 오차율이 22%p 줄어들었고, 2번보다도 4%p 더 좋은 성능을 보여줍니다.

이로써 사람의 도메인 지식이 추가되는 편이 더 좋은 성능의 모델을 찾을 수 있다는 것을 알 수 있습니다!

# 4. Submission

이제 완성된 모델을 이용해서 제출해봐요!

## test set 전처리

지난 번 baseline과는 다르게, 이번엔 test set에 대해서도 전처리를 진행해야 합니다.

학습한 모델은 77개의 변수를 보고 결과값을 예측하도록 배웠는데 테스트 할 때는 11개 변수만 주어선 안되겠죠.


In [17]:
test = pd.read_csv('data/test.csv') # 데이터를 불러온 뒤
submission  = pd.read_csv('data/sample_submission.csv')

# check_missing_col(test) # 결측치 확인

year, month, day = seperate_datetime(test) #날짜 숫자로 분리 후 

#새로운 컬럼을 생성
test['year'] = year 
test['month'] = month
test['day'] = day

# date_time은 제거합니다.
test_X = test.drop('date_time', axis = 1)

In [18]:
# 요일 정보 추가

week_day = pd.to_datetime(test['date_time']).dt.day_name()
le = LabelEncoder()
le.fit(week_day)
test_X['week_day'] = le.transform(week_day)

# 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
test_X['temp_diff_info'] = test_X['high_temp'] - test_X['low_temp']

# 2. "덥고 습한 날씨"를 알기 위한 정보
test_X['sweat_info'] = test_X['high_temp'] * test_X['humidity'] 

# 3. "춥고 바람부는 날씨"를 알기 위한 정보
test_X['cold_info'] = test_X['low_temp'] * test_X['wind_speed'] 

test_X

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day,week_day,temp_diff_info,sweat_info,cold_info
0,108.833,3.000,0.000,2.900,28.333,11.800,20.667,18.333,4,1,1,4,8.867,585.558111,34.220000
1,116.717,3.850,0.000,2.662,46.417,12.000,19.000,28.500,4,1,2,0,7.000,881.923000,31.944000
2,82.669,4.000,0.565,2.165,77.258,8.875,16.368,52.847,4,1,3,2,7.493,1264.558944,19.214375
3,44.123,3.466,0.466,3.747,63.288,6.250,17.368,37.671,4,1,4,3,11.118,1099.185984,23.418750
4,147.791,1.500,0.000,1.560,48.176,7.188,18.684,4.459,4,1,5,1,11.496,900.120384,11.213280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,177.149,3.980,0.223,1.066,74.628,20.312,28.579,36.486,4,3,26,2,8.267,2132.793612,21.652592
87,138.723,2.777,0.135,1.290,70.236,20.812,29.000,18.378,4,3,27,3,8.188,2036.844000,26.847480
88,111.095,3.338,1.270,1.692,70.338,21.000,28.789,35.946,4,3,28,1,7.789,2024.960682,35.532000
89,171.622,3.270,0.595,1.470,70.473,21.000,29.421,27.770,4,3,29,5,8.421,2073.386133,30.870000


In [19]:
# 두 변수간의 곱 변수를 추가합니다.
col_list = test_X.columns

for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        test_X[f'{col_list[i]}*{col_list[j]}'] = test_X[col_list[i]] * test_X[col_list[j]]

In [20]:
# train data로 학습시킨 모델에 test_X데이터를 넣고 예측합니다.
test_yhat = model.predict(test_X)

# submission dataFrame 완성
submission['number_of_rentals'] = test_yhat

# 제출 파일 생성
submission.to_csv('211109_submission.csv', index= False)

이제 여러분의 제출 파일이 생성되었습니다.

결과물을 제출하고 여러분의 점수를 확인해보세요!

# 결론

방금 보았듯이 Feature Engineering 하나만으로도 엄청난 성능 향상을 만들어 낼 수 있습니다.

또한 단순히 컴퓨터가 자동으로 계산해주는 것보다, 사람의 판단이 같이 들어갔을 때 더욱 뛰어난 성능을 보여준다는 것을 알 수 있었습니다.

이 때문에 데이터 분석에서는 분석 대상에 대한 지식(domain knowledge)과 EDA를 통한 인사이트 도출이 중요한 것입니다.

여러분은 이번 데이터에서 어떠한 인사이트를 얻을 수 있었나요?

여러분의 코드를 코드공유에 올려주세요!

좋아요와 댓글을 합산하여 가장 높은 점수를 얻으신 분께 데이콘 후드가 제공됩니다!